# Carga del modelo

In [2]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import numpy as np


In [3]:
model = gensim.models.Word2Vec.load("./doc2vec_wikipedia_es_pvdbow")
model.docvecs.vectors_docs.shape
model.most_similar(positive=['seguros'],topn=10)


C:\Users\Carla\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


[('aseguradora', 0.6253476738929749),
 ('reaseguros', 0.5982040166854858),
 ('aseguradoras', 0.5916717052459717),
 ('rga', 0.5637242794036865),
 ('bancarios', 0.5485397577285767),
 ('asegurador', 0.5484205484390259),
 ('bancaria', 0.5317405462265015),
 ('crédit', 0.5307000875473022),
 ('reaseguradoras', 0.5265746116638184),
 ('banca', 0.5258691906929016)]

# Carga de Data

In [6]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

bdtesis = client.BDTesis
documentos = bdtesis.documentosSBS


In [7]:
#a minusculas
data = []
for x in documentos.find():
    data.append(x['contenido'])
datamin =[]
for x in (data):
    datamin.append(x.lower()) 
data


['El presente Reglamento es aplicable a las empresas señaladas en el literal D del artículo 16º de la Ley General del Sistema Financiero y del Sistema de Seguros y Orgánica de la Superintendencia de Banca y Seguros, Ley N° 26702 y sus normas modificatorias, en adelante empresas. ',
 'Para la aplicación del presente Reglamento se consideran las siguientes definiciones: ',
 'Actividad económica: Agrupación de actividad según el “Global Industry Classification Standard – GICS”. El grado de agrupación que debe aplicarse corresponde al nivel de “Industria” (industry), dentro de la estructura del estándar antes mencionado, identificado con códigos de seis (06) dígitos. ',
 'AFP: Administradoras Privadas de Fondos de Pensiones. ',
 'Criterios de elegibilidad: aquellos que se indican en el artículo 24°. ',
 'Grupo económico: Se determina de acuerdo a lo dispuesto en las Normas Especiales sobre Vinculación y Grupo Económico, aprobadas por la Resolución SBS Nº 5780-2015 y sus normas modificatori

In [8]:
#limpieza de puntuación
import re
import nltk
import string
cadpun = string.punctuation + '¿'
datapunc = []
for x in datamin:
    datapunc.append(x.translate(str.maketrans('', '', cadpun)))


In [9]:
#retirar stopwords
from nltk.corpus import stopwords
stops = stopwords.words('spanish')
dataclean = []
for x in datapunc:
    palabras = x.split()
    resultwords  = [palabra for palabra in palabras if palabra.lower() not in stops]
    dataclean.append(' '.join(resultwords))
    

In [10]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(dataclean)]

In [11]:
datamatrix = []
for x in dataclean:
    palabras = x.split()
    datamatrix.append(palabras)
datamatrix

[['presente',
  'reglamento',
  'aplicable',
  'empresas',
  'señaladas',
  'literal',
  'd',
  'artículo',
  '16º',
  'ley',
  'general',
  'sistema',
  'financiero',
  'sistema',
  'seguros',
  'orgánica',
  'superintendencia',
  'banca',
  'seguros',
  'ley',
  'n°',
  '26702',
  'normas',
  'modificatorias',
  'adelante',
  'empresas'],
 ['aplicación',
  'presente',
  'reglamento',
  'consideran',
  'siguientes',
  'definiciones'],
 ['actividad',
  'económica',
  'agrupación',
  'actividad',
  'según',
  '“global',
  'industry',
  'classification',
  'standard',
  '–',
  'gics”',
  'grado',
  'agrupación',
  'debe',
  'aplicarse',
  'corresponde',
  'nivel',
  '“industria”',
  'industry',
  'dentro',
  'estructura',
  'estándar',
  'mencionado',
  'identificado',
  'códigos',
  'seis',
  '06',
  'dígitos'],
 ['afp', 'administradoras', 'privadas', 'fondos', 'pensiones'],
 ['criterios', 'elegibilidad', 'aquellos', 'indican', 'artículo', '24°'],
 ['grupo',
  'económico',
  'determina'

In [12]:
vectoresPF = []
for pregunta in datamatrix:
    vector = model.infer_vector(pregunta)
    vectoresPF.append(vector)
len(vectoresPF)


613

In [13]:
#GUARDADO EN BD, SE CREA UNA COLUMNA NUEVA EN LA COLECCIÓN
cont = 0
for x in bdtesis.documentosSBS.find():
    bdtesis.documentosSBS.update({"_id": x['_id']}, {"$set": {"vector": vectoresPF[cont].tolist()}})
    cont = cont +1

C:\Users\Carla\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  after removing the cwd from sys.path.
